                               Why are students absent?

Introduction:
    New York City Public Schools are the largest public school system in the United States. As such, large datasets about all kinds of variables are created in order to facilitate the optimal education for all students. The data is publicly available and compiled by PASSNYC, a non-profit dedicated to broadening educational opportunities for New York City students. 
    In this project I am using data from this compiled dataset in order to answer the question "Why are students absent?". To answer this question, I am...

#explain why I picked these variables 

Data Cleaning:

In [76]:
#import all software needed
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
#read in data
df = pd.read_csv('2016 School Explorer.csv')

In [78]:
relevant_columns = ['Economic Need Index', 'Average ELA Proficiency', 'Average Math Proficiency', 'Trust %', 'Strong Family-Community Ties %', 'Student Attendance Rate', 'Percent of Students Chronically Absent']

#replace "N/A" to a null variable for relevant columns that contain "N/A"
df.replace('N/A', np.nan, inplace=True)

#check data for null variables in relevant columns
print(df[relevant_columns].isnull().any())

#check data for type
print(df[relevant_columns].dtypes)

Economic Need Index                       True
Average ELA Proficiency                   True
Average Math Proficiency                  True
Trust %                                   True
Strong Family-Community Ties %            True
Student Attendance Rate                   True
Percent of Students Chronically Absent    True
dtype: bool
Economic Need Index                       float64
Average ELA Proficiency                   float64
Average Math Proficiency                  float64
Trust %                                    object
Strong Family-Community Ties %             object
Student Attendance Rate                    object
Percent of Students Chronically Absent     object
dtype: object


In [79]:
#eliminate rows with null variables
df.dropna(subset=relevant_columns, inplace=True)
#might need to split it into variables with percentage and those without, also don't need to change the float variables
#convert columns to float values
relevant_columns_table = df[relevant_columns].apply(lambda x: x.str.rstrip('%').astype('float') / 100.0)
relevant_columns_table

AttributeError: Can only use .str accessor with string values!

Create Summary Statistics Tables

In [31]:
#create seperate tables for each summary statistic
mean_values = relevant_columns_table[relevant_columns].mean()
median_values = relevant_columns_table[relevant_columns].median()
mode_values = relevant_columns_table[relevant_columns].mode()
variance_values = relevant_columns_table[relevant_columns].var()
std_values = relevant_columns_table[relevant_columns].std()
skew_values = relevant_columns_table[relevant_columns].skew()
iqr_values = relevant_columns_table[relevant_columns].quantile([0.25, 0.75])
kurtosis_values = relevant_columns_table[relevant_columns].kurtosis()
#possibly add range

#create dataframes for each summary statistic 
mean_df = pd.DataFrame(mean_values, columns=['Mean'])
median_df = pd.DataFrame(median_values, columns=['Median'])
mode_df = pd.DataFrame(mode_values, columns=['Mode'])
variance_df = pd.DataFrame(variance_values, columns=['Variance'])
std_df = pd.DataFrame(std_values, columns=['Standard Deviation'])
skew_df = pd.DataFrame(skew_values, columns=['Skewness'])
iqr_df = pd.DataFrame(iqr_values, columns=['Q1', 'Q3']).transpose()
kurtosis_df = pd.DataFrame(kurtosis_values, columns=['Kurtosis'])

#merge tables to show one clean output

summary_stats = pd.concat([mean_df, median_df, mode_df, variance_df, std_df, skew_df, iqr_df, kurtosis_df], axis=1)
print(summary_stats)

NameError: name 'relevant_columns_table' is not defined